In [13]:
import os

import numpy as np
import pandas as pd
path = 'DAT-WS2425-Projektarbeit\DAT-WS2425-Projektarbeit\Daten\Strompreisdaten'
extension = '.csv'

files = [file for file in os.listdir(path) if file.endswith(extension)]

dfs = []
filecounter =0
for file in files:
    df = pd.read_csv(os.path.join(path, file))
    dfs.append(df)
    filecounter += 1
    
print("total files: ", filecounter)
df = pd.concat(dfs, ignore_index=True)


total files:  258


In [19]:
df['Datum (MEZ)'] = pd.to_datetime(df['Datum (MEZ)'])
df['Datum (MESZ)'] = pd.to_datetime(df['Datum (MESZ)'])

In [20]:
df

,Datum (MEZ),Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh),Datum (MESZ)
0,2019-12-30 00:00:00,14233.89,29930.56,11.07,NaT
1,2019-12-30 01:00:00,14154.34,30166.97,11.84,NaT
2,2019-12-30 02:00:00,13251.87,30427.25,6.16,NaT
3,2019-12-30 03:00:00,13186.11,31289.24,1.40,NaT
4,2019-12-30 04:00:00,13045.57,32203.27,4.81,NaT
...,...,...,...,...,...
43387,2024-12-10 19:00:00,NaN,NaN,180.17,NaT
43388,2024-12-10 20:00:00,NaN,NaN,166.60,NaT
43389,2024-12-10 21:00:00,NaN,NaN,151.15,NaT
43390,2024-12-10 22:00:00,NaN,NaN,137.94,NaT
